Задача:
Выполнить оценку работы библиотеки по извлечению текста новости. Необходимо определить полноту извлекаемого текста и оценить значимость тех частей текста, которые выделить не удалось. Проанализировать и предложить изменения в алгоритм работы библиотеки.
 
Желаемый результат:
Заполненная таблица -
с эталоном текста для каждой новости;
с результатами оценки полноты (сравнения эталона текста и текста, извлеченного библиотекой);
оценка значимости текста, который выделить не удалось;
комментарии, если есть предложения по улучшению.

In [1]:
# 📦 Стандартные библиотеки
import os
import re
from typing import List
from operator import itemgetter

# for .env file support and get access to api keys
from dotenv import load_dotenv
load_dotenv()

# 📦 Сторонние библиотеки
import pandas as pd
from pydantic import BaseModel, Field

# 🚀 LangChain и расширения
from langchain.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain_core.output_parsers import StrOutputParser
from langchain.output_parsers import (
    PydanticOutputParser,
    StructuredOutputParser,
    OutputFixingParser,
)
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_core.runnables import (
    Runnable,
    RunnablePassthrough,
    chain,
    RunnableConfig,
)

# 🔎 Graph Retriever
from langchain_graph_retriever import GraphRetriever
from graph_retriever.strategies import Eager
from langchain.schema import Document  # Если нужен именно этот Document
from dotenv import load_dotenv
import glob
from langchain_core.documents import Document
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_deepseek import ChatDeepSeek




In [ ]:
# import pandas as pd

# from pydantic import BaseModel, Field
# from typing import Dict, Any, List, Optional

# from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
# from langchain_core.output_parsers import PydanticOutputParser, StrOutputParser
# from langchain.output_parsers import OutputFixingParser
# from langchain_core.tools import tool
# from langchain_core.runnables import RunnablePassthrough
# from langchain.agents import initialize_agent, AgentType


# from langchain_huggingface.llms import HuggingFacePipeline
# from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# gpu_llm = HuggingFacePipeline.from_model_id(
#     model_id="../models/Mistral-7B-Instruct-v0.3",
#     task="text-generation",
#     pipeline_kwargs={"max_new_tokens": 50},
# )


# cpu_llm = HuggingFacePipeline.from_model_id(
#     model_id="../models/DeepSeek-R1-Distill-Qwen-7B",
#     task="text-generation",
#     pipeline_kwargs={"max_new_tokens": 50},
# )


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cpu


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cpu


In [2]:

template = """Question: {question}

Answer: Let's think step by step."""
prompt = PromptTemplate.from_template(template)


gpu_chain = prompt | gpu_llm

question = "What is electroencephalography?"

print(gpu_chain.invoke({"question": question}))

Question: What is electroencephalography?

Answer: Let's think step by step.

Electroencephalography (EEG) is a technique used to record the electrical activity of the brain.

Here's a breakdown:

1. 'Electro': Related to electricity.

2


In [ ]:
template = """Question: {question}

Answer: Let's think step by step."""
prompt = PromptTemplate.from_template(template)

cpu_chain = prompt | cpu_llm

question = "What is electroencephalography?"

print(cpu_chain.invoke({"question": question}))

Question: What is electroencephalography?

Answer: Let's think step by step. Electroencephalography, or EEG, is a method used to record electrical activity in the brain. It works by placing small sensors, called electrodes, on the scalp. These electrodes detect the electrical signals produced by the brain's neurons as they communicate


## Load data

In [17]:
data_test_web = pd.read_csv("tmp/data_test_web.csv")
data_test_web = data_test_web.dropna(subset=["web_text"]).reset_index(drop=True)
data_test_web.head()

,URL,lib_text,web_text
0,https://expert.ru/ekonomika/vygodna-li-rossii-...,Серьезнее других от введения западных санкций ...,Выгодна ли России отмена санкций США Ослаблени...
1,https://ria.ru/20250311/klyuchevaya_stavka-196...,"МОСКВА, 14 фев — РИА Новости Ключевая процентн...","МОСКВА, 25 апр — РИА Новости.Ключевая процентн..."
2,https://expert.ru/mnenie/denis-manturov-gosuda...,Со следующего года начнется реализация 12 мега...,Денис Мантуров: «Государство в рамках нацпроек...
3,https://ria.ru/20250304/kredity-2003042476.html,Самозапрет на кредиты - это новая возможность ...,"МОСКВА, 4 мар - РИА Новости.Самозапрет на кред..."
4,https://lenta.ru/brief/2025/03/11/green/,"Заходя в магазин, вы часто замечали зеленые уп...","11:43, 11 марта 2025\nСреда обитания\nКомпании..."


In [3]:
llm = ChatDeepSeek(
    model="deepseek-reasoner",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [4]:
llm.invoke('hello, how are you?')

AIMessage(content="Hello! 😊 I'm doing great—just a digital bundle of energy ready to help you out. How about you? How's your day going? 💬", additional_kwargs={'refusal': None, 'reasoning_content': "Okay, the user just said “hello, how are you?” Such a polite and warm opening! \n\nHmm, they're probably starting a casual conversation. Might be testing the waters or genuinely checking in. Since it's a greeting, they likely expect a friendly but brief response—no deep analysis needed here. \n\nI should match their tone: upbeat but not overbearing. Adding a smile emoji feels right to keep it light. And since they asked about me, I'll bounce the question back to show interest in them too. \n\n...Wait, is this their first message? Maybe they're new. Better keep it extra simple and inviting. No jargon, just approachable. “I'm great” sounds positive without being fake. “How about you?” turns it into a two-way street naturally. \n\nAlso noting zero urgency or frustration in their words—pure neut

## Простой подход к анализу текста

In [ ]:
class OneComment(BaseModel):
    comment: str = Field(..., description="Выведи комментарий к оценке полноты раскрытия, укажи, что раскрыто полностью, частично или не раскрыто и почему ты так думаешь")
    one_score: str = Field(..., description='''Выведи оценку полноты раскрытия ключевого момента от 0 до 100
                           , где 0 - ключевые моменты отсутствуют и/или не раскрыты, 100 - полностью раскрыты или лучше раскрыты.
                           ''')

class OutputData(BaseModel):
    key_moments: List[OneComment] = Field(..., description="Выведи ключевые моменты раскрытия темы, которые ты будешь оценивать")
    comment: str = Field(..., description="Выведи комментарий к средней оценки полноты раскрытия по всем ключевым моментам")
    overall_score: int = Field(..., description='''Выведи среднюю оценку полноты раскрытия по всем ключевым моментам от 0 до 10
                               , где 0 - ключевые моменты отсутствуют и/или не раскрыты, 100 - полностью раскрыты или лучше раскрыты.
                           ''')

parser = PydanticOutputParser(pydantic_object=OutputData)
parser = OutputFixingParser.from_llm(parser=parser, llm=llm)

In [ ]:
prompt = ChatPromptTemplate.from_template(
    '''
    Выделить ключевые моменты текста {web_text}. Определить наличие этих ключевых моментов и полноту их раскрытия от 0 до 10 в тексте {lib_text}, 
    где 0 - ключевые моменты отсутствуют и/или не раскрыты, 10 - полностью раскрыты или лучше раскрыты.
    Формируй вывод следующим образом:{template}
'''
)

llm_chain = (
    prompt
    | llm 
    | parser
)

result = llm_chain.invoke({
    'template': parser.get_format_instructions(),
    "web_text": data_test_web["web_text"][0],
    "lib_text": data_test_web["lib_text"][0]
})

result

OutputData(key_moments=[OneComment(comment='Уязвимость импортозависимых отраслей (нефтегаз, машиностроение, авиапром) раскрыта полностью: приведены статистика (48.2% в фармацевтике) и примеры компаний.', one_score='10'), OneComment(comment='Переход на китайские технологии в автопроме рассмотрен частично: упомянуты кейсы («АвтоВАЗ»), но отсутствует анализ долгосрочных последствий.', one_score='7'), OneComment(comment='Развитие МСП раскрыто поверхностно: упомянута роль, но нет конкретных механизмов или данных.', one_score='6'), OneComment(comment='Адаптация финансовой инфраструктуры (СПФС, цифровой рубль) описана неполно: глобальные альтернативы SWIFT не проанализированы.', one_score='7'), OneComment(comment='Противоречивые последствия снятия санкций раскрыты хорошо: показаны риски для производителей и исторические прецеденты (Куба, Иран).', one_score='9')], comment='Текст охватывает большинство ключевых аспектов с опорой на данные и примеры, но отдельные моменты (ВПК, долгосрочные прогн

## Подход с анализом ключевых моментов и их аспектов

In [48]:
class AspectOneKeyMoment(BaseModel):
    aspect_key_moment: str = Field(..., description="Выведи аспект ключевого момента")
    aspect_key_moment_comment: str = Field(..., description="Выведи подробный комментарий к аспекту ключевого момента")

class OneKeyMoment(BaseModel):
    # comment: str = Field(..., description="Выведи подробное описанием ключевого момента.")
    one_key_moment: str = Field(..., description='''
                            Выведи ключевой момент.
                           ''')
    all_aspects: List[AspectOneKeyMoment] = Field(..., description="Выведи все аспекты ключевого момента")
    one_key_moment_comment: str = Field(..., description="Выведи комментарий по ключевому моменту")


class AllKeyMoments(BaseModel):
    all_key_moments: List[OneKeyMoment] = Field(..., description="Выведи все ключевые моменты")
    all_key_moments_comment: str = Field(..., description="Выведи комментарий по всем ключевым моментам")


parser = PydanticOutputParser(pydantic_object=AllKeyMoments)
parser = OutputFixingParser.from_llm(parser=parser, llm=llm)

In [49]:
prompt_key_points = ChatPromptTemplate.from_template(
    '''
    Выделить ключевые моменты текста {web_text} с подробным описанием каждого ключевого момента.
    Отметь подробно все важные аспекты каждого ключевого момента.
    Формируй вывод следующим образом:{template}
'''
)

llm_chain = (
    prompt_key_points
    | llm 
    | parser

)


# web text analyze by key moments and aspects
web_key_moments = []
for text_id in range(len(data_test_web["web_text"][:2])):       
    result = llm_chain.invoke({
        'template': parser.get_format_instructions(),
        "web_text": data_test_web["web_text"][text_id]        
    })
    web_key_moments.append(result)
    print(f"Processed text_id={text_id} of {len(data_test_web['web_text'])}")

# pd.DataFrame(web_key_moments, columns=["all_key_moments", "all_key_moments_comment"]).to_csv("tmp/web_key_moments.csv", index=False)

Processed text_id=0 of 50
Processed text_id=1 of 50


In [50]:
web_key_moments

[AllKeyMoments(all_key_moments=[OneKeyMoment(one_key_moment='Отрасли, наиболее пострадавшие от санкций', all_aspects=[AspectOneKeyMoment(aspect_key_moment='Нефтегазовый сектор', aspect_key_moment_comment='Лишился сервисных услуг и западных технологий добычи, что критично для оборудования и технологических процессов.'), AspectOneKeyMoment(aspect_key_moment='Машиностроение и авиастроение', aspect_key_moment_comment='Столкнулись с ограничениями в поставках комплектующих и технологий, особенно в производстве самолетов, кораблей и локомотивов.'), AspectOneKeyMoment(aspect_key_moment='Фармацевтика и химическая промышленность', aspect_key_moment_comment='До 48.2% конечного потребления фармацевтики попало под санкции, создав дефицит критически важной продукции.'), AspectOneKeyMoment(aspect_key_moment='Базовая электроника и IT', aspect_key_moment_comment='Несмотря на эффективное замещение в IT, сохраняется зависимость от иностранных решений в высокотехнологичных сегментах.')], one_key_moment_co

In [51]:
class AspectRatio(BaseModel):
    aspect_id: int = Field(..., description="Идентификатор аспекта ключевого момента")
    aspect_ratio_comment: str = Field(..., description="""Сравни тестовый текст по аспекту ключевого момента.
                       Выведи комментарий к сравнению, что раскрыто полностью, частично или не раскрыто и почему ты так думаешь""")
    aspect_ratio: int = Field(..., description='''. 
                       На основании этого сравнения выведи полноту тестового текста 0 до 100, где 0 - аспект не раскрыт, 100 - полностью раскрыт''')
    
class OneKeyMomentRatio(BaseModel):
    all_aspects_ratio: List[AspectRatio] = Field(..., description="Выведи все аспекты ключевого момента с их полнотой раскрытия")
    one_key_moment_ratio_comment: str = Field(..., description="""Сравни тестовый текст по ключевому моменту.
                       Выведи комментарий к сравнению, что раскрыто полностью, частично или не раскрыто и почему ты так думаешь""")
    one_key_moment_ratio: int = Field(..., description='''На основании этого сравнения выведи полноту тестового текста по ключевому моменту от 0 до 100,
                       где 0 - ключевой момент не раскрыт, 100 - полностью раскрыт''')

parser = PydanticOutputParser(pydantic_object=OneKeyMomentRatio)
parser = OutputFixingParser.from_llm(parser=parser, llm=llm)

In [52]:
# под ключевым моментом понимается ключевой момент вместе с аспектами и их подробным описанием

prompt_ratio = ChatPromptTemplate.from_template(
    '''
    Проверь насколько полно в тестовом тексте: {lib_text}. раскрыт этот ключевой момент: {agg_key_moment}. Определи полноту раскрытия каждого аспекта в отдельности
    и полноту раскрытия ключевого момента как среднее значение по всем аспектам. Ответ выведи в следующем формате: {template}    
'''
)

llm_chain_ratio = (
    prompt_ratio
    | llm
    | parser
 
)

output = []
for text_id, wkm in enumerate(web_key_moments):
    df_key_moments = []

    for target_moment in wkm.all_key_moments:
        agg_key_moment = f'ключевой момент: {target_moment.one_key_moment}\n'
        agg_key_moment += f'комментарий к ключевому моменту: {target_moment.one_key_moment_comment}\n'
        agg_key_moment += 'аспекты ключевого момента:\n\n'
        for n, aspect in enumerate(target_moment.all_aspects):
            agg_key_moment += f'аспект c id={n}: {aspect.aspect_key_moment}\n'
            agg_key_moment += f'комментарий к аспекту c id={n}: {aspect.aspect_key_moment_comment}\n\n'


        result_ratio = llm_chain_ratio.invoke({
            'template': parser.get_format_instructions(),
            "lib_text": data_test_web["lib_text"].iloc[text_id],
            "agg_key_moment": agg_key_moment
        })

        aspects = []
        for aspect in result_ratio.all_aspects_ratio:
            aspects.append({
                    "aspect_id": aspect.aspect_id
                    ,"aspect_name": target_moment.all_aspects[aspect.aspect_id].aspect_key_moment
                    ,"aspect_comment": target_moment.all_aspects[aspect.aspect_id].aspect_key_moment_comment    
                    ,"aspect_ratio": aspect.aspect_ratio
                    ,"aspect_ratio_comment": aspect.aspect_ratio_comment
                })


        df_aspects = pd.DataFrame(aspects)
        df_aspects['key_moment'] = target_moment.one_key_moment
        df_aspects['key_moment_comment'] = target_moment.one_key_moment_comment
        df_aspects['key_moment_ratio'] = result_ratio.one_key_moment_ratio
        df_aspects['key_moment_ratio_comment'] = result_ratio.one_key_moment_ratio_comment

        df_key_moments.append(df_aspects)


    df_key_moments = pd.concat(df_key_moments, ignore_index=True).reset_index(drop=True)

    df_key_moments['text_id'] = text_id

    df_key_moments['text_ratio_mean'] = df_key_moments['key_moment_ratio'].mean()
    df_key_moments['text_ratio_median'] = df_key_moments['key_moment_ratio'].median()

    output.append(df_key_moments)

df_output = pd.concat(output, ignore_index=True).reset_index(drop=True)
df_output.to_csv("tmp/web_key_moments_ratio.csv", index=False)

In [54]:
df_output

,aspect_id,aspect_name,aspect_comment,aspect_ratio,aspect_ratio_comment,key_moment,key_moment_comment,key_moment_ratio,key_moment_ratio_comment,text_id,text_ratio_mean,text_ratio_median
0,0,Нефтегазовый сектор,Лишился сервисных услуг и западных технологий ...,40,Аспект раскрыт частично. Текст подтверждает по...,"Отрасли, наиболее пострадавшие от санкций",Санкции сильнее всего ударили по отраслям с вы...,55,Ключевой момент раскрыт частично (55/100). Тек...,0,86.750000,91.5
1,1,Машиностроение и авиастроение,Столкнулись с ограничениями в поставках компле...,95,Аспект раскрыт полностью. Текст детально анали...,"Отрасли, наиболее пострадавшие от санкций",Санкции сильнее всего ударили по отраслям с вы...,55,Ключевой момент раскрыт частично (55/100). Тек...,0,86.750000,91.5
2,2,Фармацевтика и химическая промышленность,До 48.2% конечного потребления фармацевтики по...,35,Аспект раскрыт частично. Текст подтверждает вы...,"Отрасли, наиболее пострадавшие от санкций",Санкции сильнее всего ударили по отраслям с вы...,55,Ключевой момент раскрыт частично (55/100). Тек...,0,86.750000,91.5
3,3,Базовая электроника и IT,"Несмотря на эффективное замещение в IT, сохран...",50,Аспект раскрыт частично. Текст отмечает эффект...,"Отрасли, наиболее пострадавшие от санкций",Санкции сильнее всего ударили по отраслям с вы...,55,Ключевой момент раскрыт частично (55/100). Тек...,0,86.750000,91.5
4,0,Исторические прецеденты,США редко полностью отменяют санкции (примеры:...,100,Аспект раскрыт полностью. Текст детально описы...,Условия отмены санкций США,Отмена возможна лишь при выполнении политическ...,100,Ключевой момент раскрыт полностью. Текст детал...,0,86.750000,91.5
5,1,Требования к отмене,"Необходимо изменение ситуации, послужившей при...",100,Аспект раскрыт полностью. Текст четко формулир...,Условия отмены санкций США,Отмена возможна лишь при выполнении политическ...,100,Ключевой момент раскрыт полностью. Текст детал...,0,86.750000,91.5
6,2,Политический инструмент,Санкции используются США как рычаг давления дл...,100,Аспект раскрыт полностью. Текст прямо указывае...,Условия отмены санкций США,Отмена возможна лишь при выполнении политическ...,100,Ключевой момент раскрыт полностью. Текст детал...,0,86.750000,91.5
7,3,Секторальный охват,17 секторов экономики России объявлены целями ...,100,Аспект раскрыт полностью. Текст точно называет...,Условия отмены санкций США,Отмена возможна лишь при выполнении политическ...,100,Ключевой момент раскрыт полностью. Текст детал...,0,86.750000,91.5
8,0,Переориентация на Китай,"АвтоВАЗ, ГАЗ, КамАЗ и другие перешли на сборку...",100,Аспект раскрыт полностью. Текст детально описы...,Адаптация автопрома,"Отрасль перестроилась на китайские платформы, ...",93,Ключевой момент раскрыт почти полностью. Детал...,0,86.750000,91.5
9,1,Инвестиции в импортозамещение,АвтоВАЗ вложил 100+ млрд руб. в замену румынск...,100,Аспект раскрыт полностью. Приведена конкретная...,Адаптация автопрома,"Отрасль перестроилась на китайские платформы, ...",93,Ключевой момент раскрыт почти полностью. Детал...,0,86.750000,91.5


In [58]:
data_test_web['text_id'] = data_test_web.index
df_output_group_id = df_output.groupby('text_id').agg({
    'text_ratio_mean': 'mean',
    'text_ratio_median': 'median'
}).reset_index()
data_test_web = data_test_web.merge(df_output_group_id, on='text_id', how='left')

data_test_web

,URL,lib_text,web_text,text_id,text_ratio_mean,text_ratio_median
0,https://expert.ru/ekonomika/vygodna-li-rossii-...,Серьезнее других от введения западных санкций ...,Выгодна ли России отмена санкций США Ослаблени...,0,86.750000,91.5
1,https://ria.ru/20250311/klyuchevaya_stavka-196...,"МОСКВА, 14 фев — РИА Новости Ключевая процентн...","МОСКВА, 25 апр — РИА Новости.Ключевая процентн...",1,91.333333,93.0
2,https://expert.ru/mnenie/denis-manturov-gosuda...,Со следующего года начнется реализация 12 мега...,Денис Мантуров: «Государство в рамках нацпроек...,2,NaN,NaN
3,https://ria.ru/20250304/kredity-2003042476.html,Самозапрет на кредиты - это новая возможность ...,"МОСКВА, 4 мар - РИА Новости.Самозапрет на кред...",3,NaN,NaN
4,https://lenta.ru/brief/2025/03/11/green/,"Заходя в магазин, вы часто замечали зеленые уп...","11:43, 11 марта 2025\nСреда обитания\nКомпании...",4,NaN,NaN
5,https://lenta.ru/articles/2025/03/09/aslan/,"20 лет назад, 8 марта 2005 года, в чеченском с...","Date: 00:01, 9 марта 2025\nCategory: Силовые с...",5,NaN,NaN
6,https://expert.ru/finance/investorov-svyazyvay...,"Говоря сухим языком определений, цифровые фина...",Инвесторов связывают блокчейном Как ЦФА меняют...,6,NaN,NaN
7,https://ria.ru/20250123/svo-1985822676.html,Поддержка военнослужащих и их семей с самого н...,"МОСКВА, 23 янв - РИА Новости.Выплаты участника...",7,NaN,NaN
8,https://expert.ru/mnenie/vse-teper-zavisit-tol...,— За последние два с половиной года из-за введ...,"Геннадий Красников: «У нас остались школы, дос...",8,NaN,NaN
9,https://ria.ru/docs/about/privacy_policy.html,Федеральное государственное унитарное предприя...,Федеральное государственное унитарное предприя...,9,NaN,NaN


In [53]:
data_test_web = pd.read_csv("tmp/data_test_web.csv")
data_test_web.head()

,URL,lib_text,web_text
0,https://expert.ru/ekonomika/vygodna-li-rossii-...,Серьезнее других от введения западных санкций ...,Выгодна ли России отмена санкций США Ослаблени...
1,https://ria.ru/20250311/klyuchevaya_stavka-196...,"МОСКВА, 14 фев — РИА Новости Ключевая процентн...","МОСКВА, 25 апр — РИА Новости.Ключевая процентн..."
2,https://expert.ru/mnenie/denis-manturov-gosuda...,Со следующего года начнется реализация 12 мега...,Денис Мантуров: «Государство в рамках нацпроек...
3,https://ria.ru/20250304/kredity-2003042476.html,Самозапрет на кредиты - это новая возможность ...,"МОСКВА, 4 мар - РИА Новости.Самозапрет на кред..."
4,https://lenta.ru/brief/2025/03/11/green/,"Заходя в магазин, вы часто замечали зеленые уп...","11:43, 11 марта 2025\nСреда обитания\nКомпании..."


In [53]:
result_ratio

OneKeyMomentRatio(all_aspects_ratio=[AspectRatio(aspect_id=0, aspect_ratio_comment='Аспект раскрыт почти полностью. В тексте детально перечислены пострадавшие отрасли (нефтегаз, машиностроение, авиапром, фармацевтика, электроника и IT), приведены примеры ограничений (сервисные услуги, технологии добычи), механизмы адаптации (переход на китайские платформы в автопроме, рост МСП в легкой промышленности). Однако электроника и IT упомянуты кратко, без конкретных примеров замещения.', aspect_ratio=85), AspectRatio(aspect_id=1, aspect_ratio_comment='Аспект раскрыт частично. Цифры из прогноза ЦМАКП (48.2% для фармацевтики, 44.7% для химической промышленности, 32.2% для авиастроения) приведены, но отсутствует анализ их актуальности на текущий момент. Нет сравнения с реальными последствиями санкций после 2022 г., что снижает глубину раскрытия.', aspect_ratio=70)], one_key_moment_ratio_comment='Ключевой момент раскрыт полно, но с некоторыми пробелами. Текст подтверждает тезис о стимулировании им

In [ ]:
df_key_moments

In [20]:
result.all_key_moments

[OneKeyMoment(one_key_moment='Влияние санкций на ключевые отрасли экономики России', all_aspects=[AspectOneKeyMoment(aspect_key_moment='Наиболее пострадавшие отрасли', aspect_key_moment_comment='Нефтегазовый сектор, машиностроение, авиастроение, фармацевтика, электроника и IT. Санкции ограничили доступ к технологиям и сервисам, что привело к необходимости импортозамещения.'), AspectOneKeyMoment(aspect_key_moment='Прогнозы аналитиков ЦМАКП', aspect_key_moment_comment='В марте 2022 г. прогнозировалось наибольшее влияние на фармацевтику (48.2%), химическую промышленность (44.7%) и авиастроение (32.2%).')], one_key_moment_comment='Санкции стимулировали развитие отечественного производства, но в ряде отраслей (станкостроение, судостроение) замещение импорта остается незавершенным.'),
 OneKeyMoment(one_key_moment='Сценарии снятия санкций США', all_aspects=[AspectOneKeyMoment(aspect_key_moment='Условия отмены', aspect_key_moment_comment='США снимают санкции только при достижении целей их введ

In [33]:
rows = []
for n, item in enumerate(result.all_key_moments):
    for aspect in item.all_aspects:
        rows.append({
            'key_moment_id': n,
            "key_moment": item.one_key_moment,
            "key_moment_comment": item.one_key_moment_comment,
            "aspect": aspect.aspect_key_moment,
            "aspect_comment": aspect.aspect_key_moment_comment
        })

df = pd.DataFrame(rows)

df

,key_moment_id,key_moment,key_moment_comment,aspect,aspect_comment
0,0,Влияние санкций на ключевые отрасли экономики ...,Санкции стимулировали развитие отечественного ...,Наиболее пострадавшие отрасли,"Нефтегазовый сектор, машиностроение, авиастрое..."
1,0,Влияние санкций на ключевые отрасли экономики ...,Санкции стимулировали развитие отечественного ...,Прогнозы аналитиков ЦМАКП,В марте 2022 г. прогнозировалось наибольшее вл...
2,1,Сценарии снятия санкций США,Полная отмена маловероятна; возможны точечные ...,Условия отмены,США снимают санкции только при достижении целе...
3,1,Сценарии снятия санкций США,Полная отмена маловероятна; возможны точечные ...,Политическая мотивация,Санкции — инструмент давления. Их смягчение во...
4,2,Трансформация автопрома,Рынок авто стагнирует из-за высоких кредитных ...,Переход на китайские платформы,"Заводы в Калининграде, Москве и Калуге перешли..."
5,2,Трансформация автопрома,Рынок авто стагнирует из-за высоких кредитных ...,Риски возврата западных брендов,Возврат иностранных производителей потребует у...
6,3,Авиапром и импортозамещение,Авиакомпании инвестируют в российское ПО и зап...,100% локализация,Планы по выпуску отечественных самолетов к 202...
7,3,Авиапром и импортозамещение,Авиакомпании инвестируют в российское ПО и зап...,Санкционные риски,"США могут смягчить санкции в обмен на уступки,..."
8,4,Роль МСП в импортозамещении,"МСП стали драйвером замещения, но зависимость ...",Рост числа предприятий,С 5.8 млн (2022 г.) до 6.3 млн (2023 г.) — мал...
9,4,Роль МСП в импортозамещении,"МСП стали драйвером замещения, но зависимость ...",Китайская конкуренция,В легкой промышленности китайские производител...
